In [1]:
import pandas as pd
import numpy as np

"""concatenate data"""
dflist = []
for i in range(1,13):
    fn = '../2016_data_new/2016_{}_new.csv'.format(i)
    df = pd.read_csv(fn, index_col=0)
    dflist.append(df)
df = pd.concat(dflist, ignore_index=True)

In [2]:
"""select all flights with ORIGIN at SAN"""
san = df.loc[df['ORIGIN']=='SAN']
print san.head(2)
print san.columns

     YEAR  MONTH  DAY_OF_MONTH  DAY_OF_WEEK     FL_DATE UNIQUE_CARRIER  \
225  2016      1             4            1  2016-01-04             F9   
227  2016      1             4            1  2016-01-04             F9   

     AIRLINE_ID ORIGIN ORIGIN_CITY_NAME ORIGIN_STATE_NM         ...           \
225       20436    SAN    San Diego, CA      California         ...            
227       20436    SAN    San Diego, CA      California         ...            

    DEP_DELAY  ARR_TIME  ARR_DELAY  ACTUAL_ELAPSED_TIME  DISTANCE  \
225      -6.0    1148.0      -17.0                133.0     853.0   
227     -15.0    2235.0      -25.0                130.0     853.0   

     CARRIER_DELAY  WEATHER_DELAY  NAS_DELAY  SECURITY_DELAY  \
225            NaN            NaN        NaN             NaN   
227            NaN            NaN        NaN             NaN   

     LATE_AIRCRAFT_DELAY  
225                  NaN  
227                  NaN  

[2 rows x 22 columns]
Index([u'YEAR', u'MONTH', u'DAY

In [3]:
"""see destination airports"""
grouped = san.groupby(['DEST'])
grp_size = grouped.size().sort_values(ascending=False)
# airports = list(grp_size.index[:25])

airports = ['ATL', 'ORD', 'DEN', 'LAX', 'DFW', 'SFO', 'PHX', 'LAS', 'IAH', 'SEA', 
            'MSP', 'MCO', 'DTW', 'BOS', 'EWR', 'CLT', 'SLC', 'JFK', 'BWI', 'LGA', 
            'MDW', 'FLL', 'SAN', 'DCA', 'PHL']
selected = grp_size.loc[grp_size.index.isin(airports)]
# print grp_size
print selected

DEST
SFO    7760
PHX    5433
DEN    4864
LAS    4440
SEA    3632
DFW    3491
ORD    3143
LAX    3017
SLC    2475
ATL    2263
JFK    2065
IAH    1729
MDW    1299
MSP    1192
EWR    1072
BOS    1032
PHL     908
BWI     794
CLT     787
DTW     786
MCO     680
FLL     195
dtype: int64


In [4]:
"""get cities"""
for ap in airports:
    origin_df = df.loc[df['ORIGIN']==ap]
    city = origin_df['ORIGIN_CITY_NAME'].unique()[0]
    print ap, city, type(city[0])

ATL Atlanta, GA <type 'str'>
ORD Chicago, IL <type 'str'>
DEN Denver, CO <type 'str'>
LAX Los Angeles, CA <type 'str'>
DFW Dallas/Fort Worth, TX <type 'str'>
SFO San Francisco, CA <type 'str'>
PHX Phoenix, AZ <type 'str'>
LAS Las Vegas, NV <type 'str'>
IAH Houston, TX <type 'str'>
SEA Seattle, WA <type 'str'>
MSP Minneapolis, MN <type 'str'>
MCO Orlando, FL <type 'str'>
DTW Detroit, MI <type 'str'>
BOS Boston, MA <type 'str'>
EWR Newark, NJ <type 'str'>
CLT Charlotte, NC <type 'str'>
SLC Salt Lake City, UT <type 'str'>
JFK New York, NY <type 'str'>
BWI Baltimore, MD <type 'str'>
LGA New York, NY <type 'str'>
MDW Chicago, IL <type 'str'>
FLL Fort Lauderdale, FL <type 'str'>
SAN San Diego, CA <type 'str'>
DCA Washington, DC <type 'str'>
PHL Philadelphia, PA <type 'str'>


In [6]:
selected = san.loc[san['DEST'].isin(airports)]
airlines = sorted(selected['UNIQUE_CARRIER'].unique())
print airlines

['AA', 'AS', 'B6', 'DL', 'F9', 'NK', 'OO', 'UA', 'VX', 'WN']


In [10]:
selected = san.loc[san['DEST'].isin(airports)]

# print selected.head()
# print selected['DEST'].unique()
# print df.loc[(df['ORIGIN']=='SAN') & (df['DEST']=='DCA')]

"""get count of delays per airlines"""
delay_ap = {}
for ap in airports:
#     pc_list = []
    delay_m = {}
    for month in range(1,13):
        month_total = selected.loc[(selected['MONTH']==month) & (selected['DEST']==ap)]
        delays = month_total.loc[month_total['ARR_DELAY']>14]
        
        num_delay = delays.groupby(['UNIQUE_CARRIER']).size()
        num_total = month_total.groupby(['UNIQUE_CARRIER']).size()
        delay_pc = num_delay.divide(num_total)
        
        al_used = list(delay_pc.index)
        
        delay_al = np.zeros((len(airlines),))
        size_al = np.zeros((len(airlines),))
        for i,al in enumerate(airlines):
            if al in al_used:
                delay_al[i] = delay_pc[al]
                size_al[i] = 20
        delay_m[month] = (delay_al, size_al)

    delay_ap[ap] = delay_m

# print delay_ap



In [12]:
"""average delay time"""
selected = san.loc[san['DEST'].isin(airports)]

# delay_al = {}
# for al in airlines:
#     delay_m = {}
#     for month in range(1,13):
#         month_total = selected.loc[(selected['MONTH']==month) & (selected['UNIQUE_CARRIER']==al)]
#         delays = month_total.loc[month_total['ARR_DELAY']>0]
        
#         delay_avg = delays.groupby(['DEST'])['ARR_DELAY'].mean()
# #         print delay_avg
# #         break
        
#         ap_used = list(delay_avg.index)
#         delay_ap = np.zeros((len(airports),))
#         for i,ap in enumerate(airports):
#             if ap in ap_used:
#                 delay_ap[i] = delay_avg[ap]
                
#         delay_m[month] = delay_ap
        
#     delay_al[al] = delay_m


"""get count of delays per airlines"""
delay_ap = {}
for ap in airports:
#     pc_list = []
    delay_m = {}
    for month in range(1,13):
        month_total = selected.loc[(selected['MONTH']==month) & (selected['DEST']==ap)]
        delays = month_total.loc[month_total['ARR_DELAY']>0]
        
        delay_avg = delays.groupby(['UNIQUE_CARRIER'])['ARR_DELAY'].mean()
        
        al_used = list(delay_avg.index)
        
        delay_al = np.zeros((len(airlines),))
        size_al = np.zeros((len(airlines),))
        for i,al in enumerate(airlines):
            if al in al_used:
                delay_al[i] = delay_avg[al]
                size_al[i] = 20
        delay_m[month] = (delay_al, size_al)

    delay_ap[ap] = delay_m
print delay_ap

{'JFK': {1: (array([ 103.44444444,    0.        ,   25.82051282,   50.25      ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ]), array([ 20.,   0.,  20.,  20.,   0.,   0.,   0.,   0.,   0.,   0.])), 2: (array([ 37.3       ,   0.        ,  42.11428571,  43.72222222,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ]), array([ 20.,   0.,  20.,  20.,   0.,   0.,   0.,   0.,   0.,   0.])), 3: (array([ 12.1 ,   0.  ,  22.55,  12.6 ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ]), array([ 20.,   0.,  20.,  20.,   0.,   0.,   0.,   0.,   0.,   0.])), 4: (array([ 35.21428571,   0.        ,  28.24      ,  21.58333333,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ]), array([ 20.,   0.,  20.,  20.,   0.,   0.,   0.,   0.,   0.,   0.])), 5: (array([ 14.125     ,   0.        ,  18.7       ,  21.66666667,
         0.        ,   0.        